The objective of this script will be to create an experiment folder with duplicate raw files renamed
to actual experiments.

In [1]:
# import required modules / libraries
import os
import glob
import numpy as np
import json
import re
from datetime import datetime
import shutil
import json

In [2]:
import errno
 
def copy(src, dest):
    try:
        shutil.copytree(src, dest)
    except OSError as e:
        # If the error was caused because the source wasn't a directory
        if e.errno == errno.ENOTDIR:
            shutil.copy(src, dest)
        else:
            print('Directory not copied. Error: %s' % e)

In [173]:
# Note the location of the raw files output from Sensing tablet
# inputfile = '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Raw/DBS910'
# inputfile ='/Volumes/MaybergStorage/DoluObatusin/projects/LFP_Analysis/raw/DBS910'
inputfile = '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908' 
# Create experiment folder if one doesn't exist
# outputfile = '/Volumes/MaybergStorage/DoluObatusin/projects/LFP_Analysis/experiments/'
# if not(os.path.exists(outputfile)):
#     os.makedirs(outputfile)

# copy raw files to experiment folder
# if os.listdir(outputfile)== []:
#     copy(inputfile, outputfile)

In [174]:
inputfile

'/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908'

In [ ]:
# shutil.copytree(basedir, temp_experiment)

In [175]:
def ins_enable_timestamps(log_file):
    pattern = re.compile("Enable Sensing")
    for i, line in enumerate(open(log_file)):
        for match in re.finditer(pattern, line):
            t_sensing = line[15:]
            datetime_object1 = datetime.strptime(t_sensing.strip(), '%m/%d/%Y %I:%M:%S %p')
#             print("Enable Sensing:",datetime_object2)

    pattern = re.compile("INS TimeStamp")
    for i, line in enumerate(open(log_file)):
        for match in re.finditer(pattern, line):
            t_ins = line[14:]
            datetime_object2 = datetime.strptime(t_ins.strip(), '%m/%d/%Y %I:%M:%S %p')
#             print("INS TimeStamp:",datetime_object1)
        #         print('Found on line %s' % (i+1))

        # Convert Enable Sensing to datetime

    return datetime_object1, datetime_object2

In [176]:
# Extract Patient ID from path
# def extract_patientid(inputfile):
#     paths = []
#     pattern = re.compile("DBS")
#     i = 0
#     for path, directories, files in os.walk(inputfile, topdown=True):
# #         print(path)
#         paths.append(path)
#         if files:
#             for match in re.finditer(pattern, path):
#                 break
#                 print(match)
#     #             print('Found on line %s' % (i+1))
#         i += 1
#     patient_id = paths[1][match.span()[0]:]
#     return patient_id

def checkid(inputfile):
    patiendID = os.path.basename(inputfile)
    print(patiendID)
    pattern = re.compile("DBS\d")
    for match in re.finditer(pattern, inputfile):
        patientIDMatch = inputfile[match.span()[0]:]
        print(patientIDMatch)
        if patientIDMatch == patiendID:
            print('Yes')
#             logfile_dict = create_logdict(inputfile)
#             pathfiles_dict, errorfiles = createpathfiles(inputfile)
#             mapfolder(inputfile,pathfiles_dict)
        else:
            print('File path must point to patient folder (i.e. DBSXXX)')
#             break

In [177]:
checkid(inputfile)

DBS908
DBS908
Yes


In [178]:
def create_logdict(inputfile):
    log_files = []
    logfile_dict = {}
    for path, directories, files in os.walk(inputfile, topdown=True):
        if files:
            for f in files:
                if f.endswith('LOG.txt'):
                    log_files.append(f)
                    logfile_dict[path] = log_files
            log_files = []
    return logfile_dict

In [122]:
logfile_dict = create_logdict(inputfile)

In [190]:
# Create a data dictionary of path and log files - only include paths that have log files
def createpathfiles(inputfile):
    logfile_dict = create_logdict(inputfile)  

    pathfiles_dict = {}
    path_files = []
    errorfiles = []
    key = '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS901/Session_2014_04_14_Monday'
    # Operate according to each path
    for key in logfile_dict:
#         print(key)

    # create a folder in path based on log file
        for k in logfile_dict[key]:
    #         print(k)
            log_file = key + '/'+ k
            if k.endswith('LOG.txt'):
    #             pass
                new_folder = log_file.strip('_LOG.txt')
                if not(os.path.exists(new_folder)):
                    os.makedirs(new_folder)
    # Get INS & Sensing start time from log file
    #         print(log_file)
            datetime_object1, datetime_object2 = ins_enable_timestamps(log_file)
    #         print(datetime_object1)
    #         print(datetime_object2)
    # Loop through path to get all files
    #     print( ' ')
            for path, directories, files in os.walk(key, topdown=True):
                if files:
#                     print(files)
                    for f in files: 
                        if f.endswith('.xml') or f.endswith('.txt'):
#                             print(f)
                            try:
                                datetime_object3 = datetime.strptime(f[6:27], '_%Y_%m_%d_%H_%M_%S_')
#                                 print(f,'--',datetime_object3)
                            except ValueError:
                                errorfiles.append(f)
                        else:
                            continue
                        
                        # compare each file's timestamp to INS & Sensing starttime and add to dictionary of {new folder: files}
                        if datetime_object1 <= datetime_object3 <= datetime_object2:
                            path_files.append( key + '/'+ f)
                            pathfiles_dict[new_folder] = path_files
                    path_files = []


    return pathfiles_dict, errorfiles

In [191]:
pathfiles_dict, errorfiles = createpathfiles(inputfile)

In [192]:
errorfiles

['908_2016_11_14_13_15_29_LOG.txt',
 '908_2016_11_14_10_54_46__MR_0.xml',
 '908_2016_11_14_11_12_45_LOG.txt',
 '908_2016_11_14_10_54_46__MR_0.txt',
 '908_2016_11_04_11_05_44__MR_21.xml',
 '908_2016_10_06_23_07_40__MR_2.txt',
 '908_2016_11_01_11_05_56__MR_19.txt',
 '908_2016_11_07_11_05_32__MR_23.xml',
 '908_2016_10_23_11_06_32__MR_13.txt',
 '908_2016_11_11_23_05_16__MR_26.xml',
 '908_2016_10_27_23_06_16__MR_16.txt',
 '908_2016_10_29_11_06_08__MR_17.txt',
 '908_2016_10_09_23_07_28__MR_4.txt',
 '908_2016_10_15_23_07_04__MR_8.txt',
 '908_2016_10_05_11_07_44__MR_1.txt',
 '908_2016_10_17_11_06_56__MR_9.xml',
 '908_2016_11_02_23_05_52__MR_20.xml',
 '908_2016_10_14_11_07_08__MR_7.txt',
 '908_2016_10_08_11_07_32__MR_3.xml',
 '908_2016_10_23_11_06_32__MR_13.xml',
 '908_2016_10_05_11_07_44__MR_1.xml',
 '908_2016_10_30_23_06_04__MR_18.xml',
 '908_2016_11_10_11_05_20__MR_25.txt',
 '908_2016_10_11_11_07_20__MR_5.txt',
 '908_2016_10_27_23_06_16__MR_16.xml',
 '908_2016_10_11_11_07_20__MR_5.xml',
 '90

In [193]:
pathfiles_dict

{'/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_11_16_06': ['/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_11_21_12_22__MR_10.xml',
  '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_11_20_27_22__MR_7.xml',
  '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_11_23_27_22__MR_19.xml',
  '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_01_12_22__MR_26.xml',
  '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_02_57_22__MR_33.txt',
  '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_05_42_22__MR_44.xml',
  '/Volumes/MaybergStorage/

## Finally move files into their respective session folders

In [189]:
pathfiles_dict

['/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_11_16_06',
 '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_12_05_45',
 '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_13_42_11',
 '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_12_Tuesday/DBS908_2016_01_12_14_35_42',
 '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_13_Wednesday/DBS908_2016_01_13_09_14_36',
 '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_13_Wednesday/DBS908_2016_01_13_10_26_35',
 '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_13_Wednesday/DBS908_2016_01_13_11_08_38',
 '/Volumes/MaybergStorage/EphysData/raw_pt_data/LFP/BR_Curated/BR/DBS908/Session_2016_01_19

In [183]:
def movefiles(pathfiles_dict):
    for key in pathfiles_dict:
        print(key)
        print('-'*40)
        for i in pathfiles_dict[key]:
            #         print(i)
            if os.path.exists(i):
                shutil.move(i, key)
            else:
                continue

In [194]:
# movefiles(pathfiles_dict)

## Get the file name for the new file to write

In [72]:
def mapfolder(inputfile,pathfiles_dict):
    patient_id = os.path.basename(os.path.dirname(inputfile)) 
    dst = inputfile + '/' + patient_id + '_' + 'pathfiles.txt'
    # Save map to text file
    # If the file name exists, write a JSON string into the file.
    if dst:
        # Writing JSON data
        with open(dst, 'w') as f:
            json.dump(pathfiles_dict, f) # use `json.loads` to do the reverse

In [73]:
mapfolder(inputfile,pathfiles_dict)